In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from scipy.stats import gamma
import numpy as np
import c3aidatalake

In [15]:
df = pd.read_csv('./rural_resurgence/outbreaklocation_by_county.csv')
df = df.iloc[:,2:]
df.to_csv('raw_outbreaklocation_by_county.csv', index = False)

In [14]:
df.to_csv?

In [18]:
raw_county_df = pd.read_csv('raw_outbreaklocation_by_county.csv')

## Rural and Underserved Counties in the United States
List from the [Consumer Finance Protection Bureau 2020 list](https://www.consumerfinance.gov/policy-compliance/guidance/mortgage-resources/rural-and-underserved-counties-list/) This is an official website of the US governement

In [19]:
ru_und_counties = pd.read_csv('cfpb_rural-underserved-list_2020.csv')

In [92]:
state_codes = ["AK","AL","AR","AZ","CA","CO","CT","DE","FL","GA","HI",'IA',"ID",'IL',"IN","KS","KY",
               "LA","MA","MD","ME","MI","MN","MO","MS","MT","NC","ND","NE","NH","NJ","NM","NV","NY",
               "OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VA","VT","WA","WI","WV","WY"]
len(state_codes)
state_dict = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'DistrictofColumbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'NewJersey': 'NJ',
    'NewMexico': 'NM',
    'NewYork': 'NY',
    'NorthCarolina': 'NC',
    'NorthDakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'RhodeIsland': 'RI',
    'SouthCarolina': 'SC',
    'SouthDakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'WestVirginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}
len(state_dict)

51

### States without underserved counties in the United States

In [175]:
us_counties = ru_und_counties.loc[ru_und_counties["State"].isin(state_codes)].copy()
[print(state_codes[i]) for i in range(50) if state_codes[i] not in list(us_counties['State'].unique())]

CT
DE
NJ
RI


[None, None, None, None]

In [176]:
us_counties

,FIPS Code,County Name,State
0,1005,Barbour County,AL
1,1011,Bullock County,AL
2,1013,Butler County,AL
3,1019,Cherokee County,AL
4,1023,Choctaw County,AL
...,...,...,...
1620,56037,Sweetwater County,WY
1621,56039,Teton County,WY
1622,56041,Uinta County,WY
1623,56043,Washakie County,WY


In [177]:
counties = [i.replace('County', '') for i in list(us_counties['County Name'])]
us_counties['counties'] = counties
us_counties

,FIPS Code,County Name,State,counties
0,1005,Barbour County,AL,Barbour
1,1011,Bullock County,AL,Bullock
2,1013,Butler County,AL,Butler
3,1019,Cherokee County,AL,Cherokee
4,1023,Choctaw County,AL,Choctaw
...,...,...,...,...
1620,56037,Sweetwater County,WY,Sweetwater
1621,56039,Teton County,WY,Teton
1622,56041,Uinta County,WY,Uinta
1623,56043,Washakie County,WY,Washakie


In [123]:
cc_df = pd.read_csv('./rural_resurgence/rural_urban_sept_oct_casecount.csv')
cc_df['state_abbrev'] = cc_df['state'].map(state_dict)

## Counties "Missing" from the C3.ai Rural Counties But in the Rural and Underserved Counties List

In [133]:
# cc_df['fips.id']=cc_df['fips.id'].astype(int)
missing_rural_counties = [(us_counties['County Name'][i], us_counties['FIPS Code'][i]) for i in range(1625) if us_counties['FIPS Code'][i] not in list(cc_df['fips.id'])]

In [178]:
indx_missing = [i for i in range(1625) if us_counties['FIPS Code'][i] not in list(cc_df['fips.id'])]
missing_rural = us_counties.iloc[indx_missing]
missing_rural

,FIPS Code,County Name,State,counties
35,2105,"Hoonah-Angoon Census Area, AK",AK,"Hoonah-Angoon Census Area, AK"
45,2198,Prince of Wales-Hyder Census Area,AK,Prince of Wales-Hyder Census Area
47,2230,Skagway Municipality,AK,Skagway Municipality
50,2158,Kusilvak Census Area,AK,Kusilvak Census Area
51,2275,Wrangell City and Borough,AK,Wrangell City and Borough
253,15005,Kalawao County,HI,Kalawao


### "Counties" in the C3.ai Data set with Null FIP Codes

In [148]:
null_df = cc_df[cc_df['fips.id'].isnull()]
for i in range(len(null_df)):
    print((null_df.iloc[i].county,null_df.iloc[i].state))

('BeaverCounty,GarfieldCounty,IronCounty,KaneCounty,Washington', 'Utah')
('BoxElderCounty,CacheCounty,Rich', 'Utah')
('CarbonCounty,EmeryCounty,Grand', 'Utah')
('DoñaAna', 'NewMexico')
('Dukes,Nantucket', 'Massachusetts')
('DukesCounty,Nantucket', 'Massachusetts')
('Kalawao', 'Hawaii')
('Kusilvak', 'Alaska')
('Skagway', 'Alaska')
('St.LouisCity', 'Missouri')
('UintahCounty,DuchesneCounty,Daggett', 'Utah')
('Valdez-CordovaCensusArea,KodiakIslandBorough,KenaiPeninsula', 'Alaska')
('WeberCounty,Morgan', 'Utah')
('WheatlandCounty', 'Montana')


In [111]:
cc_df

,county,state,fips.id,populationCDS,rural_urban,sept_2020,oct_2020,sept_percent,oct_percent,percent_change,state_abbrev
0,Abbeville,SouthCarolina,45001.0,24527.0,rural,591.0,801.0,2.409589,3.265789,0.856199,SC
1,Acadia,Louisiana,22001.0,62045.0,urban,2969.0,3169.0,4.785237,5.107583,0.322347,LA
2,Accomack,Virginia,51001.0,32316.0,rural,1182.0,1223.0,3.657631,3.784503,0.126872,VA
3,Ada,Idaho,16001.0,481587.0,urban,13129.0,17326.0,2.726195,3.597688,0.871494,ID
4,Adair,Iowa,19001.0,7152.0,rural,76.0,220.0,1.062640,3.076063,2.013423,IA
...,...,...,...,...,...,...,...,...,...,...,...
3147,Yuma,Arizona,4027.0,213787.0,urban,12730.0,13930.0,5.954525,6.515831,0.561306,AZ
3148,Yuma,Colorado,8125.0,10019.0,rural,125.0,227.0,1.247630,2.265695,1.018066,CO
3149,Zapata,Texas,48505.0,14179.0,rural,333.0,358.0,2.348544,2.524861,0.176317,TX
3150,Zavala,Texas,48507.0,11840.0,rural,388.0,558.0,3.277027,4.712838,1.435811,TX


# I decided to Only use counties that had corresponding FIPS codes as that provided by the list of rural and underserved communities by the US government
 - The ones in the US government List but not in the C3 list are all in Alaska or Hawaii, I decided to drop those

In [165]:
raw_counties = pd.read_csv('raw_counties_info.csv')

In [182]:
indx_rural = [i for i in range(len(raw_counties)) if raw_counties['fips.id'][i] in us_counties['FIPS Code']]
rural_raw_counties = raw_counties.loc[raw_counties['fips.id'].isin(list(us_counties['FIPS Code']))]
# rural.to_csv('raw_rural_counties.csv')

In [186]:
rural_raw_counties

,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,populationOfAllChildren,latestLaborForce,latestEmployedPopulation,latestUnemployedPopulation,latestUnemploymentRate,laborForceOfAllChildren,locationType,populationCDS,id,name,typeIdent,location.value.id,location.timestamp,fips.id
2,10.0,46.0,52.0,32316.0,32316.0,16001.0,15096.0,905.0,5.655897,16001.0,county,32316.0,Accomack_Virginia_UnitedStates,Accomack,EP_LOC,Accomack_Virginia_UnitedStates,2020-11-11T00:00:00Z,51001.0
4,4.0,25.0,25.0,7152.0,7152.0,3862.0,3747.0,115.0,2.977732,3862.0,county,7152.0,Adair_Iowa_UnitedStates,Adair,EP_LOC,Adair_Iowa_UnitedStates,2020-11-11T00:00:00Z,19001.0
5,NaN,NaN,NaN,19202.0,19202.0,6523.0,6164.0,359.0,5.503603,6523.0,county,19202.0,Adair_Kentucky_UnitedStates,Adair,EP_LOC,Adair_Kentucky_UnitedStates,2020-11-11T00:00:00Z,21001.0
6,10.0,51.0,93.0,25343.0,25343.0,10077.0,9776.0,301.0,2.987000,10077.0,county,25343.0,Adair_Missouri_UnitedStates,Adair,EP_LOC,Adair_Missouri_UnitedStates,2020-11-11T00:00:00Z,29001.0
7,5.0,30.0,67.0,22194.0,22194.0,8290.0,7881.0,409.0,4.933655,8290.0,county,22194.0,Adair_Oklahoma_UnitedStates,Adair,EP_LOC,Adair_Oklahoma_UnitedStates,2020-11-11T00:00:00Z,40001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3144,8.0,50.0,50.0,18010.0,18010.0,7867.0,7415.0,452.0,5.745519,7867.0,county,18010.0,Young_Texas_UnitedStates,Young,EP_LOC,Young_Texas_UnitedStates,2020-11-11T00:00:00Z,48503.0
3146,NaN,NaN,NaN,5230.0,5230.0,2261.0,2062.0,199.0,8.801415,2261.0,county,5230.0,Yukon-Koyukuk_Alaska_UnitedStates,Yukon-Koyukuk Census Area,EP_LOC,Yukon-Koyukuk_Alaska_UnitedStates,2020-11-11T00:00:00Z,2290.0
3148,6.0,30.0,30.0,10019.0,10019.0,5010.0,4868.0,142.0,2.834331,5010.0,county,10019.0,Yuma_Colorado_UnitedStates,Yuma,EP_LOC,Yuma_Colorado_UnitedStates,2020-11-11T00:00:00Z,8125.0
3150,NaN,NaN,NaN,11840.0,11840.0,3309.0,2748.0,561.0,16.953762,3309.0,county,11840.0,Zavala_Texas_UnitedStates,Zavala,EP_LOC,Zavala_Texas_UnitedStates,2020-11-11T00:00:00Z,48507.0


In [184]:
file_id = rural_raw_counties['id']

In [190]:
#getting State and County and County Equivalent names from the ['id'] string
full_names = rural_raw_counties['id'].tolist()
split_names = [str(full_names[x]).split("_") for x in range(len(full_names))]
county_names = [split_names[x][0] for x in range(len(split_names))]
state_names = [split_names[x][1] for x in range(len(split_names))]

1619

In [191]:
final_counties = pd.DataFrame({"fips_code":rural_raw_counties['fips.id'], "county": county_names, "state": state_names, "c3_data_id": file_id})
final_counties

,fips_code,county,state,c3_data_id
2,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates
4,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates
5,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates
6,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates
7,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates
...,...,...,...,...
3144,48503.0,Young,Texas,Young_Texas_UnitedStates
3146,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates
3148,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates
3150,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates


In [192]:
final_counties.to_csv('final_rural_counties.csv')